# AdventureWorks – Försäljningsanalys

Denna notebook innehåller en analys av försäljningsdata från AdventureWorks-databasen.
Data hämtas via SQL och analyseras vidare i Python med Pandas och visualiseras med Matplotlib.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

server = "localhost,1433"
database = "AdventureWorks2025"
username = "sa"
password = quote_plus("Str0ngPass!2025")
driver = quote_plus("ODBC Driver 18 for SQL Server")

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

In [4]:
with engine.connect() as conn:
    print(conn.execute(text("SELECT DB_NAME()")).scalar())

C:\Users\SG199\AppData\Local\Temp\ipykernel_3132\4109320759.py:1: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  with engine.connect() as conn:


AdventureWorks2025
